In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.externals import joblib
from collections import Counter
from scipy import signal
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
data = joblib.load('data.pkl')

In [3]:
data_dir = 'bbdc_2019_Bewegungsdaten/'

In [4]:
data.keys()

dict_keys(['features', 'labels', 'bounds', 'lens', 'label_encoder'])

In [5]:
def standard_scale(data):
    features = data['features']
    lens = np.concatenate((data['lens']['train'],data['lens']['valid'],data['lens']['test']))
    features = np.concatenate(features, axis=0)
    features = StandardScaler().fit_transform(features)
    ret = []
    l = 0
    for t in lens:
        ret.append(features[l: l + t])
        l += t
    features = np.array(ret)
    data['features'] = features

In [6]:
def filter_emg(x):
    high = 20/(1000/2)
    low = 450/(1000/2)
    b, a = signal.butter(4, [high, low], btype='bandpass')
    emg_filtered = signal.filtfilt(b, a, x, axis=0)
    return emg_filtered

In [7]:
# standard_scale(data)

In [8]:
def get_split(data, split):
    if split == 'train':
        return data['features'][: data['bounds']['train']]
    elif split == 'valid':
        return data['features'][data['bounds']['train']: data['bounds']['train'] + data['bounds']['valid']]
    elif split == 'test':
        return data['features'][data['bounds']['train'] + data['bounds']['valid']:]

In [9]:
def generate_batch_idx(n, batch_size, randomise=False):
    idx = np.arange(0, n)
    if randomise:
        np.random.shuffle(idx)
    for batch_idx in np.arange(0, n, batch_size):
        yield idx[batch_idx:batch_idx+batch_size]

In [10]:
def generate_batches(data, split, batch_size,
                     time_steps=10, stride=5, randomise=False):
    features = get_split(data, split)

    try:
        labels = data['label_encoder'].transform(data['labels'][split])
    except:
        labels = np.zeros(features.shape[0])
    
    lens = data['lens'][split]
    new_features = []
    new_labels = []
    new_lens = []
    for i in range(len(lens)):
        mat = features[i]
        label = labels[i]
        l = lens[i]
        acc_emg = [0, 1, 2, 3, 5, 6, 7, 9, 10, 11]
        mat[:, acc_emg] -= mat[:, acc_emg].mean(axis=0)
        mat[:, :4] = filter_emg(mat[:, :4])
        extracted_steps = []
        for j in range(0, len(mat) - time_steps, stride):
            window = mat[j: j + time_steps, :]
            means = window[:, 4:].mean(axis=0).reshape(1, -1)
            rms = np.sqrt((window[:, :4]**2).mean(axis=0)).reshape(1, -1)
            feature_vector = np.concatenate((rms, means), axis=1).reshape(1, -1)
            
            extracted_steps.append(feature_vector)
        extracted_steps = np.concatenate(extracted_steps, axis=0)
        new_features.append(extracted_steps)
        new_labels.append(label)
        new_lens.append(len(extracted_steps))
    features = np.array(new_features)
    labels = np.array(new_labels)
    lens = np.array(new_lens)
    
    n = len(features)
    for batch_idx in generate_batch_idx(n, batch_size, randomise):
        batch_data = features[batch_idx]
        batch_labels = labels[batch_idx]
        batch_lens = lens[batch_idx]
#         batch_data = torch.from_numpy(batch_data).float()
#         batch_labels = torch.from_numpy(labels[batch_idx]).float()
#         lens = torch.from_numpy(lens[batch_idx]).float()
        yield batch_data, batch_labels, batch_lens

In [11]:
# def sort_batch(batch, targets, lengths):
#     """
#     Sort a minibatch by the length of the sequences with the longest sequences first
#     return the sorted batch targes and sequence lengths.
#     This way the output can be used by pack_padded_sequences(...)
#     """
#     perm_idx = np.argsort(lengths)[::-1]
#     seq_lengths = lengths[perm_idx]
#     seq_tensor = batch[perm_idx]
#     target_tensor = targets[perm_idx]
#     return seq_tensor, target_tensor, seq_lengths

def pad_batch(batch, lens):
    max_len = max(lens)
    batch_size = batch.shape[0]
    num_feature = batch[0].shape[1]
    padded_seqs = np.zeros((batch_size, max_len, num_feature))
    
    for i, l in enumerate(lens):
        padded_seqs[i, :l, :] = batch[i][:l]

    return padded_seqs

In [12]:
def torch_batch(batch, targets):
    return torch.from_numpy(batch).float(), torch.from_numpy(targets).long()

In [13]:
def get_preds(model, data, split, batch_size, time_steps, stride):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for b_data, b_labels, b_lens in generate_batches(data, split, batch_size, 
                                                         time_steps, stride, False):
            b_data = pad_batch(b_data, b_lens)
            b_data, b_labels = torch_batch(b_data, b_labels)
            preds.append(model(b_data, b_lens))
            labels.append(b_labels)
        preds = torch.cat(preds, dim=0)
        labels = torch.cat(labels, dim=0)
    return preds, labels

In [14]:
def get_accuracy(preds, labels, le):
    preds = preds.max(dim=1)[1].numpy()
    preds = [le.classes_[i] for i in preds]
    labels = labels.numpy()
    labels = [le.classes_[i] for i in labels]
    return accuracy_score(labels, preds)

In [28]:
class HARNet(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.rnn1 = nn.RNN(19, 64, num_layers=1)
        self.rnn2 = nn.RNN(64, 64)
        self.lin2 = nn.Linear(64*3, 256, bias=True)
        self.lin3 = nn.Linear(256, 22, bias=True)
    
    def forward(self, data, lens):
        x, _ = self.rnn1(data)
        x = x[:, ::8, :]
        x, _ = self.rnn2(x)
        x_max = torch.cat([seq.max(dim=0)[0].view(1, -1) for seq in x], dim=0)
        x_min = torch.cat([seq.min(dim=0)[0].view(1, -1) for seq in x], dim=0)
        x_avg = torch.cat([seq.mean(dim=0).view(1, -1) for seq in x], dim=0)
#         x_max = torch.cat([seq[0:l,:].max(dim=0)[0].view(1, -1) for l, seq in zip(lens, x)], dim=0)
#         x_min = torch.cat([seq[0:l,:].min(dim=0)[0].view(1, -1) for l, seq in zip(lens, x)], dim=0)
#         x_avg = torch.cat([seq[0:l,:].mean(dim=0).view(1, -1) for l, seq in zip(lens, x)], dim=0)
        x = torch.cat((x_min, x_avg, x_max), dim=1)
        x = self.lin2(x)
        x = F.relu(x)
        x = self.lin3(x)
        return x

In [30]:
num_epochs = 10000
batch_size = 32
time_steps = 10
stride = 5
objective = nn.CrossEntropyLoss()
model = HARNet()
optimiser = torch.optim.Adam(model.parameters(), weight_decay=0.0)
running_loss = 0
running_batch = 0
min_valid_loss = float('inf')

for epoch in range(1, num_epochs + 1):
    with tqdm(enumerate(generate_batches(data, 'train', batch_size,
                                         time_steps, stride, True), 1)) as pbar:
        model.train()
        for batch_num, (batch_data, batch_labels, batch_lens) in pbar:
            batch_data = pad_batch(batch_data, batch_lens)
            batch_data, batch_labels = torch_batch(batch_data, batch_labels)
            optimiser.zero_grad()
            preds = model(batch_data, batch_lens)
            loss = objective(preds, batch_labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 4, norm_type=2)
            optimiser.step()
            running_loss += loss.item()
            running_batch += 1
            pbar.set_description(f'[Epoch: {epoch}] | Batch {batch_num} | Loss: {running_loss/running_batch}')
            
    valid_preds, valid_labels = get_preds(model, data, 'valid',
                                          64, time_steps, stride)
    
    valid_loss = objective(valid_preds, valid_labels).item()
    
    if valid_loss < min_valid_loss:
        print(f'Validation loss improved from {min_valid_loss} to {valid_loss}')
        acc = get_accuracy(valid_preds, valid_labels, data['label_encoder'])
        print(f'Validation accuracy: {acc}')
        min_valid_loss = valid_loss
        with open('best_rnn_model.pt', 'wb') as f:
            torch.save(model.state_dict(), f)
    else:
        print('Validation loss did not improve')

[Epoch: 1] | Batch 186 | Loss: 2.391327222188314: : 186it [02:36,  3.73it/s] 
0it [00:00, ?it/s]

Validation loss improved from inf to 1.9982115030288696
Validation accuracy: 0.21867881548974943


[Epoch: 2] | Batch 186 | Loss: 2.138530998140253: : 186it [02:36,  4.48it/s] 
0it [00:00, ?it/s]

Validation loss improved from 1.9982115030288696 to 1.9344723224639893
Validation accuracy: 0.2870159453302961


[Epoch: 3] | Batch 186 | Loss: 2.027135161088786: : 186it [02:39,  3.93it/s] 
0it [00:00, ?it/s]

...

[Epoch: 179] | Batch 186 | Loss: 1.626620467478086: : 186it [02:36,  4.33it/s] 
0it [00:00, ?it/s]

Validation loss did not improve


[Epoch: 180] | Batch 186 | Loss: 1.6258989760311702: : 186it [02:37,  4.46it/s]
0it [00:00, ?it/s]

Validation loss did not improve


KeyboardInterrupt: 

In [39]:
model.load_state_dict(torch.load('best_rnn_model.pt'))

In [40]:
train_preds, train_labels = get_preds(model, data, 'train',
                                          64, time_steps, stride)

/Users/nilavro/miniconda3/envs/analytics/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [41]:
train_loss = objective(train_preds, train_labels).item()
train_loss

1.5043046474456787

In [42]:
acc = get_accuracy(train_preds, train_labels, data['label_encoder'])
acc

0.43340060544904135

In [43]:
valid_preds, valid_labels = get_preds(model, data, 'valid',
                                          64, time_steps, stride)

In [44]:
valid_loss = objective(valid_preds, valid_labels).item()
valid_loss

1.6424287557601929

In [45]:
acc = get_accuracy(valid_preds, valid_labels, data['label_encoder'])
acc

0.3895216400911162